# A/B test 效应分析

In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

/Users/mengmengwang/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from IPython.display import display,HTML
display(HTML('<style>.container{width:100% !important;}</style>'))

In [3]:
df = pd.read_excel('AB_data.xlsx')

In [4]:
df

,user_id,city,enrollment_date,traffic_source,teacher_id,finish_status,renewal_status,renewal_date,call_id,call_date,call_duration,teacher_group
0,7,一线,2024-01-06,次级渠道,1000,0,0,NaT,NaN,NaT,NaN,A
1,220,四线,2024-01-06,主流渠道,1000,1,0,NaT,NaN,NaT,NaN,A
2,436,三线,2024-01-06,次级渠道,1000,1,1,2024-01-25,NaN,NaT,NaN,A
3,600,新一线,2024-01-06,主流渠道,1000,1,0,NaT,NaN,NaT,NaN,A
4,780,五线,2024-01-06,主流渠道,1000,1,1,2024-01-07,NaN,NaT,NaN,A
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39109,新一线,2024-01-07,次级渠道,1099,1,0,NaT,NaN,NaT,NaN,A
39996,39334,一线,2024-01-07,主流渠道,1099,1,0,NaT,8498.0,2024-01-13,6.0,A
39997,39554,新一线,2024-01-07,次级渠道,1099,1,0,NaT,8499.0,2024-01-11,12.0,A
39998,39744,三线,2024-01-07,主流渠道,1099,1,0,NaT,8500.0,2024-01-14,4.0,A


### 描述性统计数据

In [5]:
# 每组续报率
df_1 = df.groupby(["enrollment_date","teacher_group"]).agg(total_users=("user_id", "count"), renewed_users=("renewal_status", "sum"),finish_users = ("finish_status", "sum"),
                                                          call_users = ("call_id", "count")).reset_index()
df_1

,enrollment_date,teacher_group,total_users,renewed_users,finish_users,call_users
0,2024-01-06,A,10600,678,9521,0
1,2024-01-06,B,9400,628,8491,0
2,2024-01-07,A,10600,1465,9547,8501
3,2024-01-07,B,9400,621,8436,0


In [6]:
# 每组续报率
df_2 = df.groupby(["enrollment_date","teacher_group","city"]).agg(total_users=("user_id", "count"), renewed_users=("renewal_status", "sum"),finish_users = ("finish_status", "sum"),
                                                          call_users = ("call_id", "count")).reset_index()
df_2

,enrollment_date,teacher_group,city,total_users,renewed_users,finish_users,call_users
0,2024-01-06,A,一线,1758,105,1581,0
1,2024-01-06,A,三线,1745,108,1546,0
2,2024-01-06,A,二线,1770,123,1597,0
3,2024-01-06,A,五线,1755,120,1588,0
4,2024-01-06,A,四线,1772,109,1597,0
5,2024-01-06,A,新一线,1800,113,1612,0
6,2024-01-06,B,一线,1513,100,1380,0
7,2024-01-06,B,三线,1612,96,1446,0
8,2024-01-06,B,二线,1624,105,1461,0
9,2024-01-06,B,五线,1531,114,1400,0


In [33]:
# 构造 家访电话标志
df['call'] = 1
df.loc[df['call_id'].isnull(),'call'] = 0


# 每组续报率
df_renewal = df.groupby(["enrollment_date","teacher_group","city","traffic_source",
                         "teacher_id","finish_status","call"]).agg(total_users=("user_id", "count"), 
                                                                   renewed_users=("renewal_status", "sum")).reset_index()

df_renewal['renewal_rate'] = df_renewal['renewed_users']/df_renewal['total_users']

In [34]:
df_renewal

,enrollment_date,teacher_group,city,traffic_source,teacher_id,finish_status,call,total_users,renewed_users,renewal_rate
0,2024-01-06,A,一线,主流渠道,1000,0,0,2,0,0.000000
1,2024-01-06,A,一线,主流渠道,1000,1,0,16,2,0.125000
2,2024-01-06,A,一线,主流渠道,1004,0,0,1,0,0.000000
3,2024-01-06,A,一线,主流渠道,1004,1,0,13,0,0.000000
4,2024-01-06,A,一线,主流渠道,1005,0,0,1,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
5063,2024-01-07,B,新一线,次级渠道,1093,1,0,19,2,0.105263
5064,2024-01-07,B,新一线,次级渠道,1094,0,0,1,0,0.000000
5065,2024-01-07,B,新一线,次级渠道,1094,1,0,14,1,0.071429
5066,2024-01-07,B,新一线,次级渠道,1096,0,0,2,0,0.000000


In [35]:
# 将分组变量转换为数值
df_renewal["teacher_group_encoded"] = df_renewal["teacher_group"].map({"A": 1, "B": 0})  # 实验组为1，对照组为0

# 对分类数据进行标签编码
label_encoder = LabelEncoder()
df_renewal["city_encoded"] = label_encoder.fit_transform(df_renewal["city"])
df_renewal["traffic_source_encoded"] = label_encoder.fit_transform(df_renewal["traffic_source"])

# 试验期/历史期
df_renewal['period'] = 1
df_renewal.loc[df_renewal['enrollment_date'] == '2024-01-06','period'] = 0

In [36]:
df_renewal

,enrollment_date,teacher_group,city,traffic_source,teacher_id,finish_status,call,total_users,renewed_users,renewal_rate,teacher_group_encoded,city_encoded,traffic_source_encoded,period
0,2024-01-06,A,一线,主流渠道,1000,0,0,2,0,0.000000,1,0,0,0
1,2024-01-06,A,一线,主流渠道,1000,1,0,16,2,0.125000,1,0,0,0
2,2024-01-06,A,一线,主流渠道,1004,0,0,1,0,0.000000,1,0,0,0
3,2024-01-06,A,一线,主流渠道,1004,1,0,13,0,0.000000,1,0,0,0
4,2024-01-06,A,一线,主流渠道,1005,0,0,1,0,0.000000,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5063,2024-01-07,B,新一线,次级渠道,1093,1,0,19,2,0.105263,0,5,1,1
5064,2024-01-07,B,新一线,次级渠道,1094,0,0,1,0,0.000000,0,5,1,1
5065,2024-01-07,B,新一线,次级渠道,1094,1,0,14,1,0.071429,0,5,1,1
5066,2024-01-07,B,新一线,次级渠道,1096,0,0,2,0,0.000000,0,5,1,1


In [37]:
# 构建回归模型
model = smf.ols(
    "renewal_rate ~ period * teacher_group_encoded * call * city_encoded * traffic_source_encoded",
    data=df_renewal,
).fit()

# 输出模型结果
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           renewal_rate   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     21.24
Date:                Thu, 27 Feb 2025   Prob (F-statistic):           9.06e-71
Time:                        20:12:29   Log-Likelihood:                 3858.0
No. Observations:                5068   AIC:                            -7676.
Df Residuals:                    5048   BIC:                            -7545.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------